In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install -q kaggle
!pip install ultralytics # install yolov8
!pip install --quiet vietocr==0.3.12 # install vietocr
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.0/705.0 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.0 MB/s eta 0:00:00


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/CS114/Pipeline/Material/kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d chithinguyen/vietnamese-book-covers
!unzip /content/vietnamese-book-covers.zip

Streaming output truncated to the last 5000 lines.
  inflating: crop_images/crop_img/IMG_7484_crop_2.jpg  
  inflating: crop_images/crop_img/IMG_7484_crop_3.jpg  
  inflating: crop_images/crop_img/IMG_7484_crop_4.jpg  
  inflating: crop_images/crop_img/IMG_7484_crop_5.jpg  
  inflating: crop_images/crop_img/IMG_7484_crop_6.jpg  
  inflating: crop_images/crop_img/IMG_7485_crop_0.jpg  
  inflating: crop_images/crop_img/IMG_7485_crop_1.jpg  
  inflating: crop_images/crop_img/IMG_7485_crop_2.jpg  
  inflating: crop_images/crop_img/IMG_7485_crop_3.jpg  
  inflating: crop_images/crop_img/IMG_7485_crop_4.jpg  
  inflating: crop_images/crop_img/IMG_7485_crop_5.jpg  
  inflating: crop_images/crop_img/IMG_7485_crop_6.jpg  
  inflating: crop_images/crop_img/IMG_7487_crop_0.jpg  
  inflating: crop_images/crop_img/IMG_7487_crop_1.jpg  
  inflating: crop_images/crop_img/IMG_7487_crop_2.jpg  
  inflating: crop_images/crop_img/IMG_7487_crop_3.jpg  
  inflating: crop_images/crop_img/IMG_7487_crop_4.jpg

In [ ]:
# Format
!mkdir /content/YOLO_data/train_images/images
!mkdir /content/YOLO_data/train_images/labels
%cd /content/YOLO_data/train_images/
!mv *.txt labels
!mv *.jpg images
!mv *.JPG images

!mkdir /content/YOLO_data/test_images/images
!mkdir /content/YOLO_data/test_images/labels
%cd /content/YOLO_data/test_images/
!mv *.txt labels
!mv *.jpg images
!mv *.JPG images

!mkdir /content/YOLO_data/val_images/images
!mkdir /content/YOLO_data/val_images/labels
%cd /content/YOLO_data/val_images/
!mv *.txt labels
!mv *.jpg images
!mv *.JPG images

%cd /content/YOLO_data/
!rm data.yaml
!echo "train: /content/YOLO_data/train_images" >> data.yaml
!echo "val: /content/YOLO_data/val_images" >> data.yaml
!echo "test: /content/YOLO_data/test_images" >> data.yaml
!echo "nc: 4" >> data.yaml
!echo "names: ['other', 'author', 'title', 'publisher']" >> data.yaml
%cd /content/

/content/YOLO_data/train_images
/content/YOLO_data/test_images
/content/YOLO_data/val_images
/content/YOLO_data
/content


In [ ]:
!rm /content/OCR_data/train_label.txt
!cp /content/drive/MyDrive/CS114/Dataset/train_label.txt /content/OCR_data/

In [ ]:
!rm -r /content/crop_images
!unzip /content/drive/MyDrive/CS114/Dataset/crop_images.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/crop_images/crop_img/IMG_7484_crop_0.jpg  
  inflating: /content/crop_images/crop_img/IMG_7484_crop_1.jpg  
  inflating: /content/crop_images/crop_img/IMG_7484_crop_2.jpg  
  inflating: /content/crop_images/crop_img/IMG_7484_crop_3.jpg  
  inflating: /content/crop_images/crop_img/IMG_7484_crop_4.jpg  
  inflating: /content/crop_images/crop_img/IMG_7484_crop_5.jpg  
  inflating: /content/crop_images/crop_img/IMG_7484_crop_6.jpg  
  inflating: /content/crop_images/crop_img/IMG_7485_crop_0.jpg  
  inflating: /content/crop_images/crop_img/IMG_7485_crop_1.jpg  
  inflating: /content/crop_images/crop_img/IMG_7485_crop_2.jpg  
  inflating: /content/crop_images/crop_img/IMG_7485_crop_3.jpg  
  inflating: /content/crop_images/crop_img/IMG_7485_crop_4.jpg  
  inflating: /content/crop_images/crop_img/IMG_7485_crop_5.jpg  
  inflating: /content/crop_images/crop_img/IMG_7485_crop_6.jpg  
  inflating: /content/crop_images/crop_

In [ ]:
# Stage 1 libraries definition
from ultralytics import YOLO
import cv2
import os
from ultralytics import YOLO
import imghdr

# Stage 2 libraries definition
import matplotlib.pyplot as plt
from PIL import Image

from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg

# Stage 3 libraries definition
from shapely.geometry import box
import pandas as pd

In [ ]:
os.makedirs('/content/Dataset', exist_ok=True)
!rm /content/Dataset/data.yaml
!echo "train: /content/YOLO_data/train_images" >> /content/Dataset/data.yaml
!echo "val: /content/YOLO_data/val_images" >> /content/Dataset/data.yaml
!echo "test: /content/YOLO_data/test_images" >> /content/Dataset/data.yaml
!echo "nc: 4" >> /content/Dataset/data.yaml
!echo "names: ['other', 'author', 'title', 'publisher']" >> /content/Dataset/data.yaml

rm: cannot remove '/content/Dataset/data.yaml': No such file or directory


# STAGE 1: Text Detection

In [ ]:
# Load a pretrained YOLOv8n model
model = YOLO('/content/drive/MyDrive/CS114/Pipeline/Material/best.pt')

# Define path to the images folder
folder_path = '/content/YOLO_data/test_images/images/'

In [ ]:
#Evaluation
metrics = model.val()
metrics #xem các hình thức đánh giá trong folder /content/runs/detect/val/

Ultralytics YOLOv8.1.6 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 16.5MB/s]
val: Scanning /content/YOLO_data/val_images/labels... 132 images, 0 backgrounds, 0 corrupt: 100%|██████████| 132/132 [00:00<00:00, 341.77it/s]

val: WARNING ⚠️ /content/YOLO_data/val_images/images/20231229_124645.jpg: corrupt JPEG restored and saved
val: New cache created: /content/YOLO_data/val_images/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:27<00:00,  3.00s/it]


                   all        132       1442      0.666      0.636      0.653      0.368
                 other        132        564      0.651       0.47      0.544      0.273
                author        132        232      0.637      0.578       0.62       0.32
                 title        132        469        0.8      0.868      0.866      0.607
             publisher        132        177      0.577      0.627      0.582      0.274
Speed: 3.2ms preprocess, 20.2ms inference, 0.0ms loss, 24.5ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d8cae663d30>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

# STAGE 2: TEXT RECOGINITION


In [ ]:
config = Cfg.load_config_from_file('/content/drive/MyDrive/CS114/Pipeline/Material/config.yml')
config['cnn']['pretrained']=False
config['device'] = 'cuda:0'
config['predictor']['beamsearch']=False
config['weights'] = '/content/drive/MyDrive/CS114/Pipeline/Material/transformerocr.pth'

In [ ]:
detector = Predictor(config)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
#này data crop_images còn sai nhiều quá nên chưa evaluate được
#train_crop_imgs.txt: line 361, 1158, 1414, 1681, 1900, 1908, 2333, 2709, 3457, 4277, 5409, 5755, 5867, 6185, 7089, 7546, 7692
#test_crop_imgs.txt: line 181, 348-363, 449-462, tobecontinued...


# STAGE 3: INFORMATION SYNTHESIS

In [ ]:
def get_bbox_center_xyxy(bbox):
    center_x = (bbox[0] + bbox[2]) / 2
    center_y = (bbox[1] + bbox[3]) / 2
    return center_x, center_y

In [ ]:
data = {'FileName': [], 'Title': [], 'Author': [], 'Publisher': [], 'Other': []}
df_predict = pd.DataFrame(data)

os.makedirs('/content/results', exist_ok=True)
os.makedirs('/content/Crop/images', exist_ok=True)

for filename in os.listdir(folder_path):
  other = []
  author = []
  title = []
  publisher = []
  file_path = os.path.join(folder_path, filename)
  results = model(file_path ) #detection
  r_sorted_boxes_by_y = sorted(results[0].boxes, key=lambda box: get_bbox_center_xyxy(box.xyxy[0])[1])
  for i, box in enumerate(r_sorted_boxes_by_y):
      img = cv2.imread(file_path)
      f = results[0].path.split('/')[-1].split('.')[0]  #file name
      b = box.xyxy[0].cpu().numpy().astype(int)
      label = box.cls[0].cpu().numpy().astype(int)
      crop = img[b[1]:b[3], b[0]:b[2]]
      if(crop.shape[0] != 0 and crop.shape[1] != 0):
        cv2.imwrite('/content/Crop/images/' + f + "_crop_" + str(i) + '.jpg', crop)
      img = Image.open('/content/Crop/images/' + f + "_crop_" + str(i) + '.jpg')
      r = detector.predict(img) #reconigtion
      if label == 0:
        other.append(r)
      elif label == 1:
        author.append(r)
      elif label == 2:
        title.append(r)
      else:
        publisher.append(r)
  other = ' '.join(other)
  author = ' '.join(author)
  title = ' '.join(title)
  publisher = ' '.join(publisher)
  new_row = {'FileName': f, 'Title': title, 'Author': author, 'Publisher': publisher, 'Other': other}

  # Thêm vào DataFrame
  df_predict = df_predict.append(new_row, ignore_index=True)
  with open('/content/results/' + f + '.txt', 'w', encoding='utf-8') as file:
    # Ghi tiêu đề
    file.write('Title:\n')
    file.write('\n'.join(title))
    file.write('\n\n')

    # Ghi tác giả
    file.write('Author:\n')
    file.write('\n'.join(author))
    file.write('\n\n')

    # Ghi nhà xuất bản
    file.write('Publisher:\n')
    file.write('\n'.join(publisher))
    file.write('\n\n')

    # Ghi các mục khác
    file.write('Other:\n')
    file.write('\n'.join(other))
    file.write('\n\n')


image 1/1 /content/YOLO_data/test_images/images/IMG_7625.jpg: 608x480 4 authors, 4 titles, 1 publisher, 9.8ms
Speed: 3.1ms preprocess, 9.8ms inference, 1.3ms postprocess per image at shape (1, 3, 608, 480)



<ipython-input-37-411a6fa07fb9>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predict = df_predict.append(new_row, ignore_index=True)


image 1/1 /content/YOLO_data/test_images/images/IMG_7475.jpg: 608x480 3 authors, 4 titles, 12.0ms
Speed: 4.1ms preprocess, 12.0ms inference, 1.8ms postprocess per image at shape (1, 3, 608, 480)



<ipython-input-37-411a6fa07fb9>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predict = df_predict.append(new_row, ignore_index=True)


image 1/1 /content/YOLO_data/test_images/images/IMG_7503.jpg: 608x480 11 others, 1 author, 4 titles, 1 publisher, 12.1ms
Speed: 4.1ms preprocess, 12.1ms inference, 1.7ms postprocess per image at shape (1, 3, 608, 480)



...

<ipython-input-37-411a6fa07fb9>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predict = df_predict.append(new_row, ignore_index=True)



image 1/1 /content/YOLO_data/test_images/images/IMG_7623.jpg: 608x480 1 other, 3 authors, 4 titles, 3 publishers, 10.2ms
Speed: 3.0ms preprocess, 10.2ms inference, 1.4ms postprocess per image at shape (1, 3, 608, 480)


<ipython-input-37-411a6fa07fb9>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predict = df_predict.append(new_row, ignore_index=True)



image 1/1 /content/YOLO_data/test_images/images/IMG_7748.jpg: 608x448 17 others, 2 authors, 7 titles, 6 publishers, 10.2ms
Speed: 2.9ms preprocess, 10.2ms inference, 1.3ms postprocess per image at shape (1, 3, 608, 448)


<ipython-input-37-411a6fa07fb9>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predict = df_predict.append(new_row, ignore_index=True)
<ipython-input-37-411a6fa07fb9>:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_predict = df_predict.append(new_row, ignore_index=True)


In [ ]:
df_predict

,FileName,Title,Author,Publisher,Other
0,IMG_7625,500 BẢI TẬP cơ bản và nâng cac HÓA HỌC 12,HUỲNH VĂN UT HUỲNH VĂN TT Giảo viên bồi dưỡng ...,NHÀ XUẤT BẢN ĐỒNG NAI,
1,IMG_7475,Giáo dục tỉnh thức,Daniel Rechtschaffen Kiều Anh Tú dịch Kiều Anh...,,
2,IMG_7503,Làm chủ KIẾN THỨC TOÁN bằng sơ đồ tư duy,Th.S Lê Văn Tiến (Chủ biên) - Hoàng Viết Thuận,NHÀ XUẤT BẢN HỒNG ĐỨC,Theo sách giảa kho th BẢN TH Kooks bộ kết nối ...
3,20231229_124914,TH BOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO...,,,Phong Trần dịch A.... 3-3: Nghệ thuật đọc vị t...
4,IMG_7758,THE TRUE IELTS Guide,PHẠM HỒNG LONG (CHỦ BIÊNI - Nguyễn Thu Thủy - ...,NHÀ XUẤT BẢN HỒNG ĐỨC,Học IELTS từ con số O chưa bao giờ tiện lợi đế...
...,...,...,...,...,...
127,IMG_7512,Bồi dưỡng LÀM VĂN HAY 8,,,Theo chương trình GDPT mới Dùng chung cho các ...
128,IMG_7458,QUẢN TRỊ CÔNG TY THỰC CHIẾN BẢN ÁN VÀ BÌNH LUẬN,LUẬT SƯ. TS. PHẠM HOÀI HUẤN,NHÀ XUẤT BẢN NHÀ XUẤT BẢN CHA SỰ THÀ CHÍNH TRỊ...,ST
129,2023_12_30_12_27_IMG_5706,DALE CARNEGIE BẬC THẦY CỦA NGHỆ THUẬT GIAO TIẾP,Trương Tú Anh Phùng Chấn Dực Người dịch: Tuệ Văn,,Trương Tú Anh Phùng Chấn Dực Người dịch: Tuệ V...
130,IMG_7623,VỞ BÀI TẬP Toán 8 8 TẬP MỘT,LÊ TUẤN ANH - ĐỖ TIẾN ĐẠT - NGUYỄN SƠN HA LÊ T...,CÔNG TY CỔ PHẦN ĐẦU TƯ XUẤT BẢN - THIẾT BỊ GIÁ...,Cánh Diều


In [ ]:
import pandas as pd

false = False
true = True

# Đọc dữ liệu từ file
with open('/content/OCR_data/test_label.txt', 'r', encoding='utf-8') as file:
    data = file.read().split('\n')

# Khởi tạo danh sách để lưu dữ liệu
file_names = []
titles = []
authors = []
publishers = []
others = []

# Xử lý từng dòng dữ liệu
for line in data:

    parts = line.split('\t')
    if len(parts) == 1:
      break
    file_name = parts[0].split('/')[1].split('.')[0]
    # print (file_name)
    # nguqua = ["2023_12_29_16_01_IMG_6060", "2023_12_29_16_14_IMG_6120"]
    # if (file_name == "2023_12_29_16_06_IMG_6086" or file_name == "2023_12_30_12_28_IMG_5710" or file_name == "2023_12_29_15_52_IMG_6032" or file_name in nguqua):
    #   continue
    annotations = eval(parts[1])  # Sử dụng eval để chuyển đổi từ chuỗi JSON thành một list các từ điển
    title_dict = {}
    author_dict = {}
    publisher_dict = {}
    other_dict = {}
    for annotation in annotations:
        transcription = annotation['transcription']
        label = transcription.split('/')[1]
        value = transcription.split('/')[0]
        index = int(transcription.split('/')[-1])
        if label == 'title':
            title_dict[index] = value
        elif label == 'author':
            author_dict[index] = value
        elif label == 'publisher':
            publisher_dict[index] = value
        else:
            other_dict[index] = value

    # Sắp xếp lại thứ tự các giá trị theo thứ tự tăng dần của giá trị cuối cùng của mỗi chuỗi
    sorted_titles = ' '.join([title_dict[i] for i in sorted(title_dict)])
    sorted_authors = ' '.join([author_dict[i] for i in sorted(author_dict)])
    sorted_publishers = ' '.join([publisher_dict[i] for i in sorted(publisher_dict)])
    sorted_others = ' '.join([other_dict[i] for i in sorted(other_dict)])

    # Thêm thông tin vào danh sách
    file_names.append(file_name)
    titles.append(sorted_titles)
    authors.append(sorted_authors)
    publishers.append(sorted_publishers)
    others.append(sorted_others)

# Tạo dataframe từ danh sách thông tin
df_test = pd.DataFrame({
    'FileName': file_names,
    'Title': titles,
    'Author': authors,
    'Publisher': publishers,
    'Other': others
})

# Hiển thị dataframe
df_test


,FileName,Title,Author,Publisher,Other
0,IMG_7529,"Thực hành Kiểm tra, đánh giá năng lực LỊCH SỬ 7",TRẦN VÂN ANH (Chủ biên) NGUYỄN THỊ HOÀI PHƯƠNG,NHÀ XUẤT BẢN ĐẠI HỌC QUỐC GIA HÀ NỘI,
1,2023_12_29_13_09_IMG_5468,Các Câu Chuyện Nhân Văn Phát Triển EQ Cho Trẻ ...,Tác giả: Liêm Đông Tinh,NHÀ XUẤT BẢN PHỤ NỮ VIỆT NAM,3-8 TUỔI Dịch giả: Bích Hợp Bồi dưỡng trí tuệ ...
2,2023_12_29_16_14_IMG_6115,14 NGUYÊN TẮC BẤT BIẾN khiến SỰ NGHIỆP THĂNG T...,Đức Quần Hà Phương dịch,NHÀ XUẤT BẢN HỒNG ĐỨC,BIZBOOKS Audio books
3,20231229_124521,An nhiên giữa những bộn bề,SHARON SALZBERG Thanh Hằng dịch,NHÀ XUẤT BẢN THANH NIÊN,Tác giả có sách bán chạy nhất theo New York Ti...
4,VH_070,GIÓ LẠNH ĐẦU MÙA,thạch lam,NXB VĂN HỌC,VĂN HỌC VIỆT NAM ĐINH TỊ BOOKS vh Tập truyện n...
...,...,...,...,...,...
127,2023_12_29_16_14_IMG_6117,HIỆU ỨNG CHIM MỒI,HẠO NHIÊN - QUỐC KHÁNH,NXB KINH TẾ TP. HỒ CHÍ MINH,1 Phiên bản cập nhật
128,20231229_123951,7 Thói quen Đánh thức Tiềm năng Sáng Tạo,SUSIE PEARL Thành Báo Ngọc dịch,NHÀ XUẤT BẢN THANH NIÊN,
129,IMG_7879,202 Thành ngữ TIẾNG ANH Giao tiếp thông dụng,KIM LOAN,NHÀ XUẤT BẢN THANH NIÊN,Cat got your tongue? Snug as a bug in a rug Ca...
130,20231229_131155,Lớp học tâm lý cho NGƯỜI HƯỚNG NỘI,JAEHOON CHOI Nguyễn Thị Vân Anh,NHÀ XUẤT BẢN DÂN TRÍ,BOOKLAND TÂM LÝ HỌC ỨNG DỤNG BOOKLAND


In [ ]:
import pandas as pd
from jiwer import wer
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from statistics import mean

ground = df_test
predict = df_predict

ground = ground.sort_values('FileName', ascending=False)
predict = predict.sort_values('FileName', ascending=False)

ground = ground.replace('', np.nan, inplace=False)
predict = predict.replace('', np.nan, inplace=False)


wer_author_lst = []
wer_title_lst  = []
wer_publisher_lst = []
wer_other_lst = []
wer_lst = [wer_author_lst, wer_title_lst, wer_publisher_lst, wer_other_lst]
col = ["Author", "Title", "Publisher", "Other"]

for c in range(len(col)):
  for i in range(len(ground)):
    #print(i)
    if type(ground[col[c]].iloc[i]) != float and type(predict[col[c]].iloc[i]) != float:
      ground_truth = ground[col[c]].iloc[i]
      hypothesis = predict[col[c]].iloc[i]
      wer_lst[c].append(wer(ground_truth, hypothesis))
wer_author = mean(wer_author_lst)
wer_title = mean(wer_title_lst)
wer_publisher = mean(wer_publisher_lst)
wer_other = mean(wer_other_lst)
print("wer_author = ",round(wer_author,4) )
print("wer_title = ",round(wer_title,4) )
print("wer_publisher = ",round(wer_publisher,4) )
print("wer_other = ",round(wer_other,4) )
mean([wer_author , wer_title ,wer_publisher, wer_other])

wer_author =  0.7605
wer_title =  0.4922
wer_publisher =  0.3661
wer_other =  0.8965


0.6288311647807225